In [1]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
import requests

In [2]:
MARGINAALI = 0.49
SIIRTO = 3.0
SAHKOVERO = 2.82752

kiinteat = MARGINAALI + SIIRTO + SAHKOVERO

In [3]:
minuuttikulutus = pd.DataFrame()

data_dir = './data'
directory = os.fsencode(data_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        ajat, kulutukset = np.loadtxt(
            os.path.join(data_dir, filename), dtype='str',
            delimiter=',', skiprows=2, unpack=True, max_rows=60)
        
        ajat = np.array(
            [datetime.strptime(i.strip(), '%d/%m/%Y %H:%M') for i in ajat])
        kulutukset = np.array(
            [float(i) for i in kulutukset])
        minuuttikulutus = pd.concat(
            [minuuttikulutus, pd.DataFrame(kulutukset, index=ajat)])

minuuttikulutus.columns = ['kulutus (kWh)']

minuuttikulutus.loc[minuuttikulutus.loc[:, 'kulutus (kWh)'] < 1] = 0

varttikulutus = minuuttikulutus.resample(
    pd.Timedelta(minutes=15)).sum().div(1e3)

In [4]:
latauksen_hinta = pd.DataFrame(
    index=varttikulutus.index,
    columns=['pörssi (snt/kWh)', 'kulutus (kWh)', 'hinta (snt)'])

In [5]:
for aika in latauksen_hinta.index:
    aika_utc = aika - pd.Timedelta(hours=3)
    aika_str = aika_utc.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    hinta = requests.get(
        'https://api.porssisahko.net/v2/price.json?date=' + aika_str).json()
    latauksen_hinta.loc[aika, 'pörssi (snt/kWh)'] = hinta['price']

In [6]:
latauksen_hinta.loc[varttikulutus.index, 'kulutus (kWh)'] = (
    varttikulutus.loc[:, 'kulutus (kWh)'])
latauksen_hinta.loc[:, 'hinta (snt)'] = (
    (latauksen_hinta.loc[:, 'pörssi (snt/kWh)'] + kiinteat)
    *latauksen_hinta.loc[:, 'kulutus (kWh)'])

print(f'{latauksen_hinta.loc[:, 'hinta (snt)'].div(1e2).sum():.2f}€')

0.38€
